!git clone https://github.com/trucomanx/cnn_face_emotion.git
!mkdir -p OUTPUTS/cnn_face_emotion/cross-validation
!wget https://www.dropbox.com/s/23tutsg2m9o78bp/fer2013.zip
!unzip -n fer2013.zip
!rm fer2013.zip

In [1]:
import os, sys, platform

# Variables

In [2]:
## Seed for the random variables
seed_number=0;



## Training hyperparameters
EPOCAS=50;
BATCH_SIZE=32;

## Model of network
model_type  = 'mobilenet_v3';
#model_type = 'efficientnet_b3'
#model_type = 'inception_v3';
#model_type = 'inception_resnet_v2';
#model_type = 'resnet_v2_50';
#model_type = 'custom1'
#model_type = 'custom_inception'
#model_type = 'custom_residual1'
#model_type = 'custom_dense1'


dataset_labels_file = 'training_labels.csv';


# Bibliotecas externas

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import datetime

# Biblioteca local

In [4]:
if os.getenv("COLAB_RELEASE_TAG"):
    sys.path.append('cnn_face_emotion/library');
else:
    sys.path.append('library');

# Uso de GPU

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


# Endereços

In [6]:

## Dataset 
if platform.system()=='Linux':
    if os.getenv("COLAB_RELEASE_TAG"):
        dataset_base_dir     = 'fer2013/archive/train';
        dataset_base_test_dir= 'fer2013/archive/test';
    elif 'microsoft-standard' in platform.uname().release:
        dataset_base_dir     = '/mnt/c/Dados/Fernando/DATASET/fer2013/archive/train';
        dataset_base_test_dir= '/mnt/c/Dados/Fernando/DATASET/fer2013/archive/test';
    else:
        dataset_base_dir     = '/mnt/boveda/DATASETs/FACE-EMOTION/fer2013/archive/train';
        dataset_base_test_dir= '/mnt/boveda/DATASETs/FACE-EMOTION/fer2013/archive/test';
else:
    dataset_base_dir     = 'C:\\Dados\\Fernando\\DATASET\\fer2013\\archive\\train';
    dataset_base_test_dir= 'C:\\Dados\\Fernando\\DATASET\\fer2013\\archive\\test';

print('dataset_base_dir:',dataset_base_dir)
print('dataset_base_test_dir:',dataset_base_test_dir)

## Output
if platform.system()=='Linux':
    if os.getenv("COLAB_RELEASE_TAG"):
        output_base_dir = 'OUTPUTS/cnn_face_emotion/train';
    elif 'microsoft-standard' in platform.uname().release:
        output_base_dir = '/mnt/c/Dados/Fernando/OUTPUTS/cnn_face_emotion/train';
    else:
        output_base_dir = '/mnt/boveda/DOCTORADO2/cnn_face_emotion_free/train';
else:
    output_base_dir = 'C:\\Dados\\Fernando\\OUTPUTS\\cnn_face_emotion\\train';

print('output_base_dir:',output_base_dir)

dataset_base_dir: /mnt/boveda/DATASETs/FACE-EMOTION/fer2013/archive/train
dataset_base_test_dir: /mnt/boveda/DATASETs/FACE-EMOTION/fer2013/archive/test
output_base_dir: /mnt/boveda/DOCTORADO2/cnn_face_emotion_free/train


# Print vars

In [ ]:
print('model_type:',model_type)
print('    epochs:',EPOCAS)
print('batch-size:',BATCH_SIZE)

# Set seed of random variables


In [7]:
np.random.seed(seed_number)
tf.keras.utils.set_random_seed(seed_number);

# Loading data of dataset

In [8]:
# Load filenames and labels
train_data = pd.read_csv(os.path.join(dataset_base_dir,dataset_labels_file));
print(train_data)
# Setting labels
Y = train_data[['label']];
L=np.shape(Y)[0];


                filename      label
0      disgusted/im0.png  disgusted
1            sad/im0.png        sad
2        fearful/im0.png    fearful
3        neutral/im0.png    neutral
4          angry/im0.png      angry
...                  ...        ...
28648   happy/im7210.png      happy
28649   happy/im7211.png      happy
28650   happy/im7212.png      happy
28651   happy/im7213.png      happy
28652   happy/im7214.png      happy

[28653 rows x 2 columns]


# Setting the cross-validation split


In [9]:
from sklearn.model_selection import train_test_split
    

training_data, validation_data = train_test_split(train_data, test_size=0.2,shuffle=True, stratify=Y)



# Data augmentation configuration

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

idg    = ImageDataGenerator(rescale=1./255,
                            rotation_range = 10,
                            width_shift_range= 0.07,
                            height_shift_range= 0.07,
                            horizontal_flip=True,
                            shear_range=1.25,
                            zoom_range = [0.75, 1.25] 
                            )

idg_val= ImageDataGenerator(rescale=1./255 )

idg_test= ImageDataGenerator(rescale=1./255 )


# Creating output directory

In [11]:
#output_dir = os.path.join(output_base_dir,'holdout');
output_dir = os.path.join(output_base_dir,'holdout_'+model_type);

try: 
    os.makedirs(output_dir) 
except: 
    pass


# Create new model

In [12]:
import lib_model as mpp

model, target_size = mpp.create_model('',model_type=model_type);
model.summary()

mpp.save_model_parameters(model, os.path.join(output_dir,'parameters_stats.m'));


Loaded layer with mobilenet_v3
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1024)              1529968   
                                                                 
 dense (Dense)               (None, 32)                32800     
                                                                 
 dense_1 (Dense)             (None, 7)                 231       
                                                                 
Total params: 1,562,999
Trainable params: 1,550,887
Non-trainable params: 12,112
_________________________________________________________________


# Defining directories

In [13]:

train_data_generator = idg.flow_from_dataframe(training_data, 
                                               directory = dataset_base_dir,
                                               target_size=target_size,
                                               x_col = "filename", 
                                               y_col = "label",
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical",
                                               shuffle = True);

valid_data_generator  = idg_val.flow_from_dataframe(validation_data, 
                                                    directory = dataset_base_dir,
                                                    target_size=target_size,
                                                    x_col = "filename", 
                                                    y_col = "label",
                                                    batch_size=BATCH_SIZE,
                                                    class_mode="categorical",
                                                    shuffle = True);

test_data_generator  = idg_test.flow_from_directory(dataset_base_test_dir, 
                                                    target_size=target_size,
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True
                                                    );

Found 22922 validated image filenames belonging to 7 classes.
Found 5731 validated image filenames belonging to 7 classes.
Found 7173 images belonging to 7 classes.


# Train and validation


In [ ]:
import matplotlib.pyplot as plt


#STEPS_BY_EPOCHS=len(train_data_generator);

# COMPILE NEW MODEL
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy','categorical_crossentropy'])

# CREATE CALLBACKS
best_model_file=os.path.join(output_dir,'model.h5');
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=best_model_file, 
                                                save_weights_only=True,
                                                monitor='val_categorical_crossentropy', 
                                                save_best_only=True, 
                                                verbose=1);

log_dir = os.path.join(output_dir,"logs","fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"));
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# There can be other callbacks, but just showing one because it involves the model name
# This saves the best model
# FIT THE MODEL
history = model.fit(train_data_generator,
                    #steps_per_epoch=STEPS_BY_EPOCHS,
                    epochs=EPOCAS,
                    validation_data=valid_data_generator,
                    callbacks=[checkpoint],
                    verbose=1
                   );


mpp.save_model_history(history,os.path.join(output_dir,"historical.csv"));


Epoch 1/50
 90/717 [==>...........................] - ETA: 11:35 - loss: 1.9618 - categorical_accuracy: 0.3812 - categorical_crossentropy: 1.6241

# Evaluate best model

In [ ]:
# LOAD BEST MODEL to evaluate the performance of the model
model.load_weights(best_model_file);

results = model.evaluate(test_data_generator)
results = dict(zip(model.metrics_names,results))
print(results,"\n\n");

with open(os.path.join(output_dir,"results_testing.txt"), 'w') as f: 
    for key, value in results.items(): 
        f.write('%s=%s;\n' % (key, value));

tf.keras.backend.clear_session()


In [ ]:

POSTNAME=str(int(results['accuracy']*100000));

tmp_name='modelo_'+model_type+'_acc'+POSTNAME+'.h5';

os.rename(best_model_file,os.path.join(output_dir,tmp_name));